<a href="https://colab.research.google.com/github/OlegLaiok/Comp_Lingv/blob/homework10/%D0%9B%D0%90%D0%9110.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
!pip install pymorphy2
!pip install pyLDAvis

In [52]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import gensim.corpora as corpora
import pandas as pd
import pymorphy2
import json
from gensim.models import LdaMulticore
from multiprocessing import Process, freeze_support
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
import os

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Применим метод латентного размещения Дирихле, для тематического моделирования документов моего корпуса.

In [54]:
analyzer = pymorphy2.MorphAnalyzer()
nltk.download('stopwords')
with open("/content/drive/MyDrive/Colab Notebooks/tokens.json", "r", encoding ='utf-8') as f:
    clear_tokens = json.load(f)
normalized = [] # Удаляем стоп слова, лемматизируем и нормализируем
stops = stopwords.words("russian") + ["это",'вечерний', 'сводка','неделя','час','регион', 'сутки','время', 'сми','область', 'месяц', 'год','день','март', 'апрель', 'радиолентач', 'человек','дом', 'город', 'житель', 'страна', 'новость','случай','слово', "который", "наш", "мочь", "год", 
                                            "такой", "знать", "мы", "свой", "один", "другой", "хотеть",
                                            "человек", "всё", "весь", "очень", "думать", "нужно",
                                            "большой", "время", "использовать", "говорить", "сказать",
                                            "иметь", "сделать", "первый", "каждый", "день", "её", "ваш",
                                            "стать", "больший", "ваше", "день", "самый", "понять",
                                            "просто", "ещё", "проблема", "также", "например"]
for tokens in clear_tokens:
    #сокращаем число частей речи, для выделения больше значимых при создании тем
    tokens = [analyzer.parse(word)[0].normal_form for word in tokens if analyzer.parse(word)[0].tag.POS in ['NOUN','ADJF','ADJS','COMP','INFN','PRTF','PRTS','GRND']]
    norm_words = [w for w in tokens if w not in stops]
    normalized.append(norm_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.g

In [55]:
id2word = corpora.Dictionary(normalized)
corpus = [id2word.doc2bow(text) for text in normalized] #cоздаем корпус слов
lda_model = LdaMulticore(corpus=corpus, id2word=id2word, num_topics=20) #рприменяем LDA модель
lda_model.print_topics()

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

[(0,
  '0.007*"женщина" + 0.006*"девочка" + 0.005*"британский" + 0.005*"россия" + 0.005*"суд" + 0.004*"коронавирус" + 0.004*"власть" + 0.004*"мужчина" + 0.004*"насилие" + 0.004*"отношение"'),
 (1,
  '0.024*"вакцина" + 0.022*"коронавирус" + 0.014*"россия" + 0.010*"вакцинация" + 0.009*"спутник" + 0.005*"российский" + 0.005*"воз" + 0.004*"рф" + 0.004*"число" + 0.004*"мир"'),
 (2,
  '0.018*"коронавирус" + 0.009*"россия" + 0.008*"компания" + 0.008*"вакцина" + 0.006*"сша" + 0.005*"вакцинация" + 0.005*"москва" + 0.004*"учёный" + 0.004*"российский" + 0.004*"спутник"'),
 (3,
  '0.026*"коронавирус" + 0.023*"новый" + 0.021*"россия" + 0.018*"московский" + 0.017*"москва" + 0.017*"санкт-петербург" + 0.017*"зафиксировать" + 0.017*"летальный" + 0.017*"исход" + 0.014*"нижегородский"'),
 (4,
  '0.013*"россия" + 0.009*"сша" + 0.008*"коронавирус" + 0.006*"власть" + 0.005*"работа" + 0.004*"дело" + 0.004*"российский" + 0.004*"компания" + 0.004*"вакцина" + 0.004*"американский"'),
 (5,
  '0.007*"дело" + 0.005

In [56]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.112121  0.058663       1        1  11.245623
15    -0.134385 -0.043565       2        1   8.236534
13    -0.048963 -0.074327       3        1   8.025275
3     -0.082564 -0.131810       4        1   6.598665
14    -0.085405  0.055273       5        1   6.333410
1     -0.084480  0.045491       6        1   5.433102
12    -0.036370  0.067238       7        1   4.902335
10     0.061687 -0.105041       8        1   4.681566
4      0.003942  0.009521       9        1   4.558974
8      0.058454  0.006613      10        1   4.325175
9      0.031258  0.062936      11        1   4.197289
2     -0.033600  0.000358      12        1   4.082096
17     0.015086  0.031650      13        1   4.021525
19     0.040331 -0.014114      14        1   3.984359
16     0.125009 -0.013253      15        1   3.983824
5      0.045244  0.064204      16        1   3.593816
0      0.082739 -0.000316      17        1   3.551641
7      0.075884  0.022151      18        1   3.216315
11     0.054600 -0.016644      19        1   2.842449
18     0.023655 -0.025028      20        1   2.186029, topic_info=                Term        Freq       Total Category  logprob  loglift
249            исход   40.000000   40.000000  Default  30.0000  30.0000
250        летальный   40.000000   40.000000  Default  29.0000  29.0000
248    зафиксировать   39.000000   39.000000  Default  28.0000  28.0000
775    нижегородский   35.000000   35.000000  Default  27.0000  27.0000
75       коронавирус  261.000000  261.000000  Default  26.0000  26.0000
..               ...         ...         ...      ...      ...      ...
250        летальный    1.191777   40.503666  Topic20  -5.6767   0.2971
248    зафиксировать    1.185363   39.766859  Topic20  -5.6821   0.3101
249            исход    1.180351   40.512435  Topic20  -5.6864   0.2873
92   санкт-петербург    1.200560   47.290906  Topic20  -5.6694   0.1496
182       московский    1.170992   47.473798  Topic20  -5.6943   0.1208

[1477 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
2566     13  0.712716      -30°с
2892     14  0.676044  10-летний
4157      7  0.658846  18-летний
3997     16  0.725958     1980-й
5115     10  0.714864  21-летний
...     ...       ...        ...
605      18  0.053304     япония
1602      9  0.205023   японский
1602     12  0.205023   японский
1602     13  0.205023   японский
1602     16  0.205023   японский

[3351 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 16, 14, 4, 15, 2, 13, 11, 5, 9, 10, 3, 18, 20, 17, 6, 1, 8, 12, 19])

Попробуем сделать то же самое, но изменить количество тем до 10 и до 30

In [63]:
lda_model = LdaMulticore(corpus=corpus, id2word=id2word, num_topics=10)
lda_model.print_topics()

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

[(0,
  '0.008*"ребёнок" + 0.004*"путин" + 0.003*"россиянин" + 0.003*"последний" + 0.003*"навальный" + 0.003*"родитель" + 0.003*"мужчина" + 0.003*"территория" + 0.003*"новый" + 0.003*"российский"'),
 (1,
  '0.036*"коронавирус" + 0.010*"вакцина" + 0.009*"вакцинация" + 0.009*"россия" + 0.008*"новый" + 0.007*"спутник" + 0.006*"москва" + 0.006*"власть" + 0.004*"россиянин" + 0.004*"ограничение"'),
 (2,
  '0.011*"коронавирус" + 0.008*"вакцина" + 0.006*"вакцинация" + 0.005*"российский" + 0.004*"рф" + 0.004*"песок" + 0.004*"россия" + 0.003*"препарат" + 0.003*"япония" + 0.003*"группа"'),
 (3,
  '0.019*"коронавирус" + 0.015*"россия" + 0.008*"новый" + 0.008*"вакцина" + 0.006*"москва" + 0.006*"московский" + 0.006*"санкт-петербург" + 0.005*"нижегородский" + 0.005*"зафиксировать" + 0.005*"исход"'),
 (4,
  '0.017*"коронавирус" + 0.014*"россия" + 0.012*"вакцина" + 0.007*"россиянин" + 0.006*"сша" + 0.004*"американский" + 0.004*"новый" + 0.004*"компания" + 0.004*"российский" + 0.003*"тест"'),
 (5,
  '0.0

In [64]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.078073  0.010778       1        1  19.090102
4      0.037320 -0.051977       2        1  12.796172
3      0.065710  0.040220       3        1  12.555145
8      0.024611  0.028223       4        1  10.132079
9     -0.006874  0.088064       5        1   8.975226
2     -0.014945 -0.106605       6        1   8.408821
7     -0.008230 -0.001098       7        1   8.342750
6      0.017244 -0.037436       8        1   7.977958
5     -0.058444  0.018688       9        1   6.741983
0     -0.134466  0.011143      10        1   4.979762, topic_info=               Term        Freq       Total Category  logprob  loglift
217         ребёнок   40.000000   40.000000  Default  30.0000  30.0000
775   нижегородский   33.000000   33.000000  Default  29.0000  29.0000
182      московский   44.000000   44.000000  Default  28.0000  28.0000
248   зафиксировать   37.000000   37.000000  Default  27.0000  27.0000
75      коронавирус  261.000000  261.000000  Default  26.0000  26.0000
...             ...         ...         ...      ...      ...      ...
1892       владелец    1.550510    7.457010  Topic10  -6.2369   1.4292
405      вакцинация    2.082133   66.243772  Topic10  -5.9421  -0.4602
1306          закон    1.617981   10.787729  Topic10  -6.1943   1.1026
78            место    1.734075   25.273482  Topic10  -6.1250   0.3205
27             дело    1.655737   35.113744  Topic10  -6.1712  -0.0546

[777 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
1197     10  0.745118  17-метровый
1958      4  0.479723   автокредит
495       4  0.297715      адвокат
495       5  0.297715      адвокат
495       7  0.148858      адвокат
...     ...       ...          ...
605       9  0.053274       япония
1602      1  0.181090     японский
1602      2  0.543269     японский
1602      5  0.181090     японский
4500      8  0.709830         яшин

[1721 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 4, 9, 10, 3, 8, 7, 6, 1])

In [59]:
lda_model = LdaMulticore(corpus=corpus, id2word=id2word, num_topics=30)
lda_model.print_topics()

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

[(11,
  '0.016*"москва" + 0.008*"россия" + 0.007*"новый" + 0.006*"коронавирус" + 0.006*"суд" + 0.006*"российский" + 0.006*"рубль" + 0.005*"санкт-петербург" + 0.005*"готовый" + 0.005*"число"'),
 (29,
  '0.007*"вакцина" + 0.006*"правительство" + 0.005*"компания" + 0.005*"коронавирус" + 0.005*"спутник" + 0.004*"местный" + 0.004*"российский" + 0.004*"заблокировать" + 0.004*"родитель" + 0.004*"стоимость"'),
 (5,
  '0.009*"язык" + 0.008*"традиционный" + 0.007*"космос" + 0.006*"русский" + 0.004*"выпуск" + 0.004*"польша" + 0.004*"альбом" + 0.004*"заказ" + 0.004*"бэйс-продюсер" + 0.004*"преступник"'),
 (2,
  '0.010*"женщина" + 0.010*"концерт" + 0.006*"лентач" + 0.006*"полицейский" + 0.005*"ребёнок" + 0.005*"москва" + 0.005*"глава" + 0.005*"акция" + 0.005*"подробность" + 0.005*"пост"'),
 (23,
  '0.005*"вакцина" + 0.005*"навальный" + 0.005*"концерт" + 0.005*"история" + 0.005*"редактор" + 0.005*"дело" + 0.005*"песня" + 0.004*"полиция" + 0.004*"мужчина" + 0.004*"сотрудник"'),
 (16,
  '0.014*"россия

In [60]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
28    -0.154506  0.009803       1        1  11.286293
21    -0.097020  0.049390       2        1   5.205312
0     -0.048531  0.083060       3        1   4.438401
3     -0.096760  0.062484       4        1   4.314417
9     -0.022004 -0.081329       5        1   4.296542
27    -0.056015  0.045728       6        1   4.275794
13    -0.066218  0.009709       7        1   4.128336
6     -0.038042 -0.096507       8        1   4.109310
1      0.075950 -0.051012       9        1   3.859738
4     -0.014307  0.044201      10        1   3.351808
25    -0.012432 -0.123972      11        1   3.244732
20    -0.073537 -0.094673      12        1   3.179323
19    -0.014200 -0.013619      13        1   3.079715
11     0.031090 -0.101906      14        1   2.980410
23     0.051804  0.069026      15        1   2.973719
16     0.058422  0.069542      16        1   2.894264
22     0.010115  0.039548      17        1   2.827805
8     -0.013538 -0.088130      18        1   2.822356
15    -0.050135  0.022774      19        1   2.756168
26     0.095230 -0.009728      20        1   2.753177
10    -0.012455  0.064099      21        1   2.749803
2      0.107153 -0.051346      22        1   2.730183
18     0.047833 -0.077908      23        1   2.666416
14     0.103894  0.044046      24        1   2.624134
17    -0.028553 -0.023954      25        1   2.393342
29     0.036442  0.037502      26        1   2.224537
24    -0.015569  0.023664      27        1   1.828224
12    -0.016546  0.040204      28        1   1.736385
7      0.118750  0.042775      29        1   1.504598
5      0.093685  0.056529      30        1   0.764753, topic_info=             Term        Freq       Total Category  logprob  loglift
75    коронавирус  264.000000  264.000000  Default  30.0000  30.0000
182    московский   42.000000   42.000000  Default  29.0000  29.0000
155        москва   64.000000   64.000000  Default  28.0000  28.0000
250     летальный   36.000000   36.000000  Default  27.0000  27.0000
249         исход   36.000000   36.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
847     известный    0.433594    8.575241  Topic30  -5.6375   1.8888
897        оборот    0.433594    5.983561  Topic30  -5.6375   2.2487
997      писатель    0.433594    5.972369  Topic30  -5.6375   2.2506
998        шедевр    0.433594    2.526469  Topic30  -5.6375   3.1109
1041    последний    0.433594   11.768915  Topic30  -5.6375   1.5723

[2067 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
1197     29  0.766632  17-метровый
4507     11  0.648701       1994-м
3234     26  0.694272    70-летний
447      20  0.711436    85-летний
4745     27  0.746870     dvd-диск
...     ...       ...          ...
1602     17  0.207512     японский
1602     18  0.207512     японский
1602     23  0.207512     японский
1602     29  0.207512     японский
175      17  0.720066    ярославль

[4462 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[29, 22, 1, 4, 10, 28, 14, 7, 2, 5, 26, 21, 20, 12, 24, 17, 23, 9, 16, 27, 11, 3, 19, 15, 18, 30, 25, 13, 8, 6])

Вывод: основываясь на данных графиках. можно сказать, что при 10 темах можно четче определить отдельные темы, не пересикающиеся друг с другом, однако при 30 темах интересно посмотреть какие темы близки между собой и какие слова в них отличаются. Однако, заметно, что на данном корпусе текстов LDA работает не очень хорошо, во многих темах встречаются слова, которые там по логике не должны быть. Я уверен, что данная проблема напрямую связана с размером датасета. Для датасета примерно в 2-3 раза больше этого результаты будут намного точнее.